In [4]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF_score: {f_score}\n')
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



In [ ]:
!pip install datasets

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('fever', 'v1.0', split='train')
df = pd.DataFrame(dataset)[:100]

# Choose target and features
y = df.label
df_features = ['claim']
X = df[df_features]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Encode labels
label_encoder = preprocessing.LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Tokenize and pad sequences
max_len = 100  # Adjust as needed
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train['claim'])
X_train_seq = tokenizer.texts_to_sequences(X_train['claim'])
X_test_seq = tokenizer.texts_to_sequences(X_test['claim'])

X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Build an RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=5, validation_split=0.2)

# Evaluate the model
y_pred_probs = model.predict(X_test_padded)
y_pred = y_pred_probs.argmax(axis=1)

# Decode predictions
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Print statistics
print_statistics(y_test, y_pred_decoded)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Epoch 1/5
2/2 [==============================] - 2s 417ms/step - loss: 1.0814 - accuracy: 0.6833 - val_loss: 1.0642 - val_accuracy: 0.6000
Epoch 2/5
2/2 [==============================] - 0s 70ms/step - loss: 1.0457 - accuracy: 0.6833 - val_loss: 1.0313 - val_accuracy: 0.6000
Epoch 3/5
2/2 [==============================] - 0s 72ms/step - loss: 0.9931 - accuracy: 0.6833 - val_loss: 0.9903 - val_accuracy: 0.6000
Epoch 4/5
2/2 [==============================] - 0s 71ms/step - loss: 0.9209 - accuracy: 0.6833 - val_loss: 0.9504 - val_accuracy: 0.6000
Epoch 5/5
1/1 [==============================] - 0s 326ms/step
Accuracy: 0.72
Precision: 0.5184
Recall: 0.72
F_score: 0.6027906976744185

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.00      0.00      0.00         1
        REFUTES       0.00      0.00      0.00         6
       SUPPORTS       0.72      1.00      0.84        18

       accuracy                           0.72        25
      macro avg       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

(0.72, 0.5184, 0.72, 0.6027906976744185)